# Task

You are given a set of images that are classified by a model trained on Imagenet as goldfish (class 1). Your task is to compute a strip of pixels of size
10 x 100 (i.e., the first 10 rows of an image of size 100 x 100), which when pasted into *each* of the original images makes the model think it is a shark (class 2) with returned probability of at least 0.5.

Note: you can score half of the points in this task if you compute a strip that works for at least one image from the list.

Important: your code needs to compute the desired strip in at most 2 minutes - it is possible to do it in less than 10 seconds on colab with GPU.


## Data

Below you can find a piece of code that download a zipfile and displays the images. Note that here the images are in their original resolution that will be later scaled down to 100 x 100.

In [ ]:
import requests, zipfile
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

url = "https://www.mimuw.edu.pl/~cygan/goldfish.zip"
request = requests.get(url)
zipfile = zipfile.ZipFile(BytesIO(request.content))
images = []
for filename in zipfile.namelist():
  if filename.endswith('jpg'):
    ifile = zipfile.open(filename)
    img = Image.open(ifile)
    images.append(img)
    plt.imshow(img)
    plt.show()
    

## Model

We will be using the `mobilenet_v2` model, solving the classification problem on the Imagenet dataset. As far as this task is concerned you do not have to know the details of the model's architecture, we are treating it as a black box, in particular the model's weights will be frozen and cannot be changed.



In [ ]:
import sys
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
import torchvision
import torchvision.transforms as transforms
import plotly.graph_objects as go
import numpy as np
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
model.to(device)
model.eval()

## Data preprocessing

In order to evaluate the model on the given images, we first scale them down to size 100x100 (which is an arbitrary choice selected for this task), but also normalize them using the following normalization transform.


In [ ]:
normalize_colors = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])

SIZE = 100
resize = transforms.Compose([transforms.Resize((SIZE, SIZE)), 
                             transforms.ToTensor()])

## Evaluation

Below is a piece of code that takes an input a given image and a tensor of size 3x10x100 with values between 0 and 1, and:
* rescales the image to size 100x100,
* replaces the first chunk of size 10x100 by the given tensor (for all 3 colors),
* normalizes the colors of the obtained image,
* evaluates the model and returns the probabilities of the image being classified as goldfish and shark respectively.

In [ ]:
STRIP_HEIGHT = 10

def show_torch(img_tensor):
  img = img_tensor.detach().numpy().T
  img = np.swapaxes(img, 0, 1)
  plt.imshow(img)
  plt.show()

def calc_predictions(image, strip, show = False):
  assert strip.shape == (3, STRIP_HEIGHT, SIZE) 
  assert torch.ge(strip, 0.).all()
  assert torch.le(strip, 1.0).all()
  img_tensor = resize(image.convert("RGB")).to(device)
  # At this point img_tensor is of shape (3, SIZE, SIZE)
  ############# Important line - replacing part of the image ################
  img_tensor[:, :STRIP_HEIGHT, :] = strip.to(device)

  if(show):
    show_torch(img_tensor)

  img_tensor = normalize_colors(img_tensor)
  # We have to add one more dimention representing the bath.
  img_tensor = img_tensor.unsqueeze(0)
  probabilities = torch.nn.functional.softmax(model(img_tensor)[0], dim=-1)
  # Class 1 in Imagenet is a goldfish, 2 is a shark
  return probabilities[1], probabilities[2]

In [ ]:
for im in images:
  p_goldfish, p_shark = calc_predictions(im, torch.zeros((3, STRIP_HEIGHT, SIZE)))
  print(f'Probability of being a goldfish is {p_goldfish}, shark is {p_shark}.')

## TO IMPLEMENT

Here you have to implement the function that finds the strip. Please don't modify the name of the function, as later it will be automatically tested.


In [ ]:
from torch.autograd import Variable

def calculate_strip():
  strip = torch.abs(torch.rand((3, STRIP_HEIGHT, SIZE)))
  strip = strip / torch.max(strip)
  return strip

def normalize_strip(strip):
  strip = torch.sigmoid(strip)
  return strip 

strip = Variable(calculate_strip(), requires_grad=True)

optimizer = optim.Adam([strip], lr=0.1)

for n in range(400):
  optimizer.zero_grad()
  new_strip = normalize_strip(strip)
  loss = 0
  
  for im in images:
    p_goldfish, p_shark = calc_predictions(im, new_strip)
    loss += -torch.log(p_shark)# + p_goldfish * 0.2

  if (n % 30 == 0):
    print(f'Epoch: {n}, loss: {loss}')

  loss.backward()
  optimizer.step()

for im in images:
  p_goldfish, p_shark = calc_predictions(im, new_strip, show = True)
  loss += -torch.log(p_shark)
  print(f'Shark p: {p_shark}, Goldfish_p: {p_goldfish}')